# Data Review and Preprocessing

In [9]:
import tensorflow as tf
import data
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

In [19]:
from sklearn.metrics import balanced_accuracy_score, accuracy_score, f1_score
from sklearn.metrics import confusion_matrix

In [12]:
df_crossmatch = pd.read_csv('datasets/raw_data/ztf/dfcrossmatches_prioritized_v6.0.csv')

In [14]:
result = []
count = 0
for chunk in pd.read_csv('datasets/raw_data/ztf/detections.csv', chunksize=1000000):
    chunk_new = chunk[['oid', 'mjd', 'fid', 'magpsf_corr', 'sigmapsf_corr']]
    temp = pd.merge(df_crossmatch[['oid', 'classALeRCE']], chunk_new, how='right', on=['oid']).dropna(subset=['classALeRCE', 'mjd'])
    result.append(list(temp.groupby('oid')))
    count +=1
    print(count)
    if count == 5:
      break

1
2
3
4
5


In [15]:
flat_list = [item for sublist in result for item in sublist]

In [16]:
labels = []
for l in flat_list:
  labels.append(l[1]['classALeRCE'].drop_duplicates().to_numpy()[0])
classes = np.unique(labels)

print(Counter(labels).keys())
print(Counter(labels).values())

dict_keys(['LPV', 'RRL', 'DSCT', 'Ceph', 'EB/EW', 'YSO', 'EA', 'CV/Nova', 'AGN', 'Blazar', 'RSCVn', 'QSO', 'Periodic-Other', 'SNII', 'SNIbc', 'SNIa', 'NLQSO', 'SLSN', 'SNIIn', 'SNIIb', 'TDE', 'NLAGN', 'ZZ'])
dict_values([81456, 67554, 1149, 940, 36557, 1335, 5430, 1226, 3954, 1444, 778, 12948, 359, 191, 31, 480, 66, 28, 32, 3, 3, 5, 1])


In [39]:
max = 0
for item in flat_list:
    if len(item[1]['fid'] == 1) > max or len(item[1]['fid'] == 2) > max:
        max = np.max([len(item[1]['fid'] == 1), len(item[1]['fid'] == 2)])

In [41]:
max

515

In [ ]:
lc_list_green = []
labels_green = []
id_green = []

lc_list_red = []
labels_red = []
id_red = []

max_zeros = np.zeros([max,3])
for i, item in enumerate(flat_list):
  max_zeros = np.zeros([max,3])
  item_red = item[1].loc[item[1]['fid'] == 1,:]
  lc = item[['mjd','magpsf_corr','sigmapsf_corr']].to_numpy()
  max_zeros[:lc.shape[0],:] = lc
  lc_list_green.append(max_zeros)
  print(i)
  if item.empty:
    temp = 'NA'
  else:
    temp = item['classALeRCE'].reset_index().loc[0].to_numpy()[1]
    temp_id = item['oid'].reset_index().loc[0].to_numpy()[1]
  labels_green.append(temp) 
  print(i)
  id_green.append(temp_id)  